In [80]:
%%capture
!pip install bs4 requests pandas

In [127]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [82]:
url = 'https://www.amazon.in/s?k=phones&ref=nb_sb_noss_1'
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
            'Accept-Language':'en-US, en;q=0.5'})

In [83]:
# HTTP req
webpage = requests.get(url, headers=HEADERS)
webpage

<Response [200]>

In [84]:
soup = BeautifulSoup(webpage.content, 'html.parser')

In [85]:
links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
links[0].get('href')

'/sspa/click?ie=UTF8&spc=MTo1NzYyMjY4NjM3MDI0MjQ6MTY3MDY1Njk2NDpzcF9hdGY6MjAwNDU5MDE0OTcyOTg6OjA6Og&url=%2FSamsung-Mystique-Storage-Purchased-Separately%2Fdp%2FB09XJ48QPR%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dphones%26qid%3D1670656964%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1'

In [115]:
product_list = "https://www.amazon.in"+links[10].get('href')
product_list

'https://www.amazon.in/Samsung-Galaxy-Storage-6000mAh-Battery/dp/B0B4F2XCK3/ref=sr_1_10?keywords=phones&qid=1670656964&sr=8-10'

In [116]:
product1 = requests.get(product_list,headers=HEADERS)
product1

<Response [200]>

In [117]:
product_content = BeautifulSoup(product1.content, 'html.parser')

In [118]:
product_content.find('span', attrs={"id":"productTitle"}).text.strip()

'Samsung Galaxy M13 (Aqua Green, 6GB, 128GB Storage) | 6000mAh Battery | Upto 12GB RAM with RAM Plus'

In [119]:
price = product_content.find('span',attrs={"class":"a-price-whole"}).text

In [120]:
rating = product_content.find('span',attrs={"class":"reviewCountTextLinkedHistogram noUnderline"}).attrs["title"]

In [133]:
product_content.find("span",attrs={"id":"acrCustomerReviewText"}).text

'16,282 ratings'

In [134]:
def getTitle(soup):
    title = ""
    try:
        title = soup.find('span', attrs={"id":"productTitle"}).text.strip()
    except:
        title = ""
    return title

def getPrice(soup):
    try:
        price = soup.find('span',attrs={"class":"a-price-whole"}).text
    except:
        price=""
    return price

def getRating(soup):
    try:
        rating = soup.find('span',attrs={"class":"reviewCountTextLinkedHistogram noUnderline"}).attrs["title"]
    except:
        rating = ""
    return rating

def getReviewcount(soup):
    try:
        count = soup.find("span",attrs={"id":"acrCustomerReviewText"}).text
    except:
        count = ""
    return count

In [137]:
if __name__=='__main__':
    url = 'https://www.amazon.in/s?k=ps5&crid=2TKYYQXVBLGWU&sprefix=ps5%2Caps%2C441&ref=nb_sb_noss_1'
    
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
            'Accept-Language':'en-US, en;q=0.5'})
    
    webpage = requests.get(url, headers=HEADERS)
    
    soup = BeautifulSoup(webpage.content, 'html.parser')
    
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    
    product_links = []
    for link in links:
        product_links.append("https://www.amazon.in"+link.get('href'))
    
    d = {"title":[], "price":[], "rating":[], "review":[]}
    for link in product_links:
        product = requests.get(link,headers=HEADERS)
        content = BeautifulSoup(product.content,'html.parser')
        
        d['title'].append(getTitle(content))
        d['price'].append(getPrice(content))
        d['rating'].append(getRating(content))
        d['review'].append(getReviewcount(content))

        amazon_df = pd.DataFrame.from_dict(d)
        amazon_df['title'].replace('', np.nan,inplace=True)
        amazon_df = amazon_df.dropna(subset=['title'])
        amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [138]:
amazon_df

,title,price,rating,review
0,God Of War Ragnarok | Launch Edition | PS5 Gam...,"4,999.",4.5 out of 5 stars,82 ratings
1,Call Of Duty : Modern Warfare II | Standard Ed...,"4,999.",3.9 out of 5 stars,7 ratings
2,Sony DualSense Wireless Controller for PlaySta...,"5,098.",4.5 out of 5 stars,"2,301 ratings"
3,PS5 God of War: Ragnarok Collector's Edition,"13,499.",4.1 out of 5 stars,4 ratings
4,PS5 Grand Theft Auto V,"2,213.",4.4 out of 5 stars,98 ratings
5,Horizon Forbidden West | Standard Edition | PS...,"2,684.",4.7 out of 5 stars,603 ratings
6,PS5 WWE 2K22,"2,299.",4.2 out of 5 stars,51 ratings
7,FIFA 23 | Standard Edition | PS5 (PlayStation 5),"4,110.",4.6 out of 5 stars,38 ratings
8,Cricket 22 | International Edition | PS5 (Play...,"2,176.",3.9 out of 5 stars,92 ratings
9,Sony Marvel's Spider-Man: Miles Morales (PS5),"2,268.",4.5 out of 5 stars,"2,609 ratings"
